## Load fine-tuned model

In [6]:
import sys, os
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.1"
OUTPUT_DIR = os.getcwd()+"/../../models/rlhf_step1_sft/"
device_map="auto"


In [7]:
from peft import PeftModel, PeftConfig
config = PeftConfig.from_pretrained(OUTPUT_DIR)

In [8]:
from transformers import AutoTokenizer
import sys, os
sys.path.append(os.getcwd()+"/../../")
from pykoi.chat.llm.instruct_pipeline import END_KEY, INSTRUCTION_KEY, RESPONSE_KEY, INTRO_BLURB
tokenizer_ft = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer_ft.pad_token = tokenizer_ft.eos_token
tokenizer_ft.add_special_tokens({"additional_special_tokens": [END_KEY, INSTRUCTION_KEY, RESPONSE_KEY+"_NL"]})

3

In [9]:
from transformers import AutoModelForCausalLM
model_ft = AutoModelForCausalLM.from_pretrained(  
    config.base_model_name_or_path,  
    #load_in_8bit=True,
    device_map=device_map,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.78s/it]


In [10]:
model_ft = PeftModel.from_pretrained(
    model_ft, 
    OUTPUT_DIR, 
    device_map=device_map,
)

model_ft.resize_token_embeddings(len(tokenizer_ft))

Embedding(32003, 4096)

## Load data

In [11]:
INPUT_FILE = os.getcwd()+'/../../data/chapter22_valfromseed_data_processed.csv'
OUTPUT_FILE = os.getcwd()+'/../../data/d2lai_selfinstruct_mistral7b-loraft_1123.csv'

In [12]:
import pandas as pd
import time
import csv

df_results = pd.read_csv (INPUT_FILE)
prompt_list = df_results.instruction.values.tolist()
reference_list = df_results.response.values.tolist()

In [13]:

len(prompt_list)


660

## Create instruct pipeline


In [38]:
from pykoi.chat.llm.instruct_pipeline import InstructionTextGenerationPipeline
import time
response_list = []
question_list = []
generate_text = InstructionTextGenerationPipeline(model=model_ft, tokenizer=tokenizer_ft)

st = time.time()

i=0
for question in prompt_list:
    #print(i," ",end = '')
    
    print(i,"-",question, end='')
    res = generate_text(question)
    response = res[0]["generated_text"]
    question_list.append(question)
    response_list.append(response)
    i = i+1

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
import pandas as pd
OUTPUT_QR_FILE = os.getcwd()+'/../../data/question_generated_answer.csv'

df = pd.DataFrame(list(zip(question_list, response_list)), columns =['Question', 'Answer'])

df.to_csv(OUTPUT_QR_FILE, index=False)


0 - What does linear algebra study?1 - Can you explain the concepts of linear algebra?2 - What are the two ways to visualize vectors in geometry?3 - How can vectors be represented using lines and arrows in geometry?4 - How do column vectors differ from row vectors?5 - Can you explain the distinction between column and row vectors?6 - What is the process of representing vectors as points in a three-dimensional space?7 - Can you explain how to graph vectors using coordinate systems and point representations?8 - Can you represent vectors using arrows on a flat surface?9 - How can vectors be depicted as arrows within a two-dimensional space?10 - What is the process of representing vector addition graphically?

/opt/conda/envs/pykoi/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


11 - How can vectors be depicted when added together using diagrams?12 - What is the process of representing vector subtraction visually?13 - How can vectors be depicted in a way that shows their difference when they are subtracted?14 - How do you calculate the inner product of two row matrices?15 - What is the formula for finding the product of two vectors with their components aligned vertically?16 - How does the dot product of two vectors relate to their magnitudes and directions in geometry?17 - Can you explain how the dot product of two vectors can be represented graphically using points and lines?18 - How do you calculate the dot product of two vectors using a formula?19 - Can you explain the formula used to determine the dot product between two vectors on page 3?20 - What is the formula to determine the angle between two vectors using their dot product and magnitudes?21 - Can you explain how to compute the angle between two vectors based on their dot product and lengths?22 - Wha

In [41]:
response_list[:10]

['Linear algebra is a branch of mathematics that deals with the study of lines, planes, and subspaces. It focuses on topics such as vector and matrix operations, linear equations, inequalities, and systems of equations. Linear algebra is used in various fields, including engineering, physics, and computer science.</s>',
 'Linear algebra is a branch of mathematics that deals with linear equations, matrices, and vector spaces. It is concerned with the study of geometric objects and the relationships between them, as well as with the use of linear transformations to model real-world phenomena. Linear algebra is an important tool in',
 'In geometry, vectors can be visualized in two ways: as lines with arrowheads or as points connected by line segments.</s>',
 "In geometry, vectors can be represented using lines and arrows to indicate both the direction and magnitude of a vector. The length of the line represents the vector's magnitude, while the arrowhead points in the direction of the vec

In [42]:
len(response_list)

660

## Evaluation with Rouge score

In [43]:
!pip install rouge_score
from rouge_score import rouge_scorer

import evaluate
rouge = evaluate.load('rouge')

import numpy as np

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
question_list[:10]

['What does linear algebra study?',
 'Can you explain the concepts of linear algebra?',
 'What are the two ways to visualize vectors in geometry?',
 'How can vectors be represented using lines and arrows in geometry?',
 'How do column vectors differ from row vectors?',
 'Can you explain the distinction between column and row vectors?',
 'What is the process of representing vectors as points in a three-dimensional space?',
 'Can you explain how to graph vectors using coordinate systems and point representations?',
 'Can you represent vectors using arrows on a flat surface?',
 'How can vectors be depicted as arrows within a two-dimensional space?']

In [45]:
def calculate_rouge(pred_list,ref_list):
    
    rouge_results = rouge.compute(predictions=pred_list,
                         references=ref_list,
                         use_aggregator=True)
    avg_rougeLsum = np.mean(rouge_results["rougeLsum"])
    avg_rougeL = np.mean(rouge_results["rougeL"])
    avg_rouge2 = np.mean(rouge_results["rouge2"])
    avg_rouge1 = np.mean(rouge_results["rouge1"])
    
    print("Average rouge score - ", avg_rougeLsum, avg_rougeL, avg_rouge2, avg_rouge1)
    
    return 

In [46]:
ref_list = reference_list
response_list = response_list
calculate_rouge(response_list, ref_list)  

Average rouge score -  0.18049280219016767 0.1787935353783797 0.09000321609423866 0.23790046057149594


## Evaluation with semantic similarity score

In [47]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
model = SentenceTransformer('thenlper/gte-large')    # use this one in production


In [49]:
def calculate_semantic_sim(pred_list,ref_list):
    
    sem_score = []
    average_sem_sim = 0
    
    for i in range(len(ref_list)):
        
        ref_embedding = model.encode(ref_list[i])
        pred_embedding = model.encode(pred_list[i])
        cos_sim = util.cos_sim(ref_embedding, pred_embedding)
        #print(cos_sim[0][0].item())
        
        sem_score.append(cos_sim[0][0].item())
    
    average_sem_sim = np.mean(sem_score)   
    
    #print("Average similarity: ", average_sem_sim)
    
    return average_sem_sim

In [50]:
avg_score = calculate_semantic_sim(reference_list,response_list)
avg_score

0.8862407925454053